<a href="https://colab.research.google.com/github/alexanderkersten/eae-dsaa/blob/main/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup

- seaborn (0.13.1)
- pydotplus (2.0.2)
- scikit-learn (1.2.2)
- pandas (2.0.3)
- dtreeviz (2.2.2)
- graphviz (0.20.3)
- matplotlib (3.7.1)

In [ ]:
!pip install dtreeviz

In [ ]:
import sklearn.datasets as datasets

iris = datasets.load_iris(as_frame=True)

# 2. Analysis

In [ ]:
type(iris)

In [ ]:
print(iris.DESCR)

In [ ]:
import pandas as pd

X = iris.data
y = iris.target

If you notice the name of the variables above, you'll see the X's and y's. Y is that?

In math notation, we define the dataset X as the array of feature vectors, each vector representing the description of a flower in our dataset, each feature representing an aspect of a flower). With all these values in X, we want to infer an approximate function $\hat{y}$, using the labeled dataset in variable `y` to better **fit** the data in `X`.

Or $\hat{y} = f(X)$

Here we also use 'y' as the name of the variable that holds the target variable (species)

In [ ]:
X.head()

In [ ]:
y.to_numpy()

In [ ]:
y.hist()

In [ ]:
iris.target_names

In [ ]:
iris_df = X.copy()
iris_df.head()

In [ ]:
species_dict = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
iris_df['species'] = y.map(species_dict)
iris_df.head()

In [ ]:
import seaborn as sns

sns.pairplot(iris_df)

In [ ]:
sns.pairplot(iris_df, hue='species', plot_kws=dict(alpha=0.3))

# 3. Model training

## Dividing the dataset into train and test datasets

We are going to create a decision tree with a partition between train and test of the data. Look at the variable *random_state* that is applied in the `train_test_split` function. By changing this variable, the random distribution between the train and test data will change.

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=70)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

## Why dividing the dataset is important?

You want your model to generalize well, so sampling is important and you need to avoid data leakage, i.e. using the training dataset as the test dataset

Additional resources

- [K Fold Cross Validation](https://www.section.io/engineering-education/how-to-implement-k-fold-cross-validation/)

## Training a Decision Tree Classifier

In decision trees we're creating structures like:
![](https://miro.com/blog/wp-content/uploads/2021/12/decision_tree_business_analysis.png)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Create the model
dtree = DecisionTreeClassifier()

# Train the model
# X_train contains the feature vectors for examples of flowers
# y_train contains the classes of these flowers
dtree.fit(X_train, y_train)

## Exporting model to file

This can be used to create a flower classifier application, for example

In [ ]:
import pickle

# Saves file (serialized dtree object)
# wb stands for Write Binary
pickle.dump(dtree, open('dtree_iris.pkl', 'wb'))

In [ ]:
# Just showing the file was actually created
!ls | grep pkl

## Visualizing resulting decision tree

In [ ]:
import pydotplus
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

In [ ]:
dot_data = StringIO()
export_graphviz(dtree,
                out_file=dot_data,
                feature_names=iris.feature_names,
                class_names=iris.target_names,
                filled=True,
                rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

**Decision Trees** or CART (Classification and Regression Trees):

This method is based on the order of the data. It is, therefore, an iterative method in which each step will try to reduce the *impurity* of the data based on the variable to be predicted. Digging a bit:

1. We start with all our data in the same bag, therefore I have all the different categories mixed together.
By categories we mean the class, the target variable, i.e. what we're trying to predict in a classification task.

    1. The impurity of this data bag is modeled using the **Gini impurity index** (not to be confused with the Gini coefficient).
    1. Simplifying it, it could be seen as: If we introduce a new observation into our bag, whose response variable has been chosen based on the distribution of the different categories, what is the probability of being wrong if I try to classify it?
    1. In other terms: if I have a sack with two pears and two oranges, the impurity is 50%, since a new variable could be with the same probability either pear or orange.

2. Then, as we want to reduce this impurity, we will distribute the observations in different bags in each iteration based on the characteristics of our variables until we obtain groups where there is no possible error: Either everything is pears or everything is oranges.


We can see that there is a variable that strongly marks a first cut, so that already in the first split we obtain a partition without impurity. It is a model capable of obtaining a very accurate division of the data in very few steps.

In [ ]:
import dtreeviz
import matplotlib as mpl

viz = dtreeviz.model(dtree, X_train, y_train,
               target_name="Species",
               feature_names=iris.feature_names,
               class_names=list(iris.target_names))

viz.view(fontname="DejaVu Sans")

In [ ]:
# dtree.feature_names_in_ = None

If we now repeat this process with a different distribution between train and test, which implies that the model has been trained with other data, by changing `random_state` to other value ...

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=60)

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

viz = dtreeviz.model(dtree, X_train, y_train,
               target_name="Species",
               feature_names=iris.feature_names,
               class_names=list(iris.target_names))

viz.view(fontname="DejaVu Sans")

In [ ]:
# Let's check the influence of the random_state variable
help(train_test_split)

## Some words about overfitting and underfitting

Sometimes you train a ML algorithm on the labeled dataset you have, but when faced with new unseen data the model starts performing bad, and therefore making wrong classifications. This happens because the model hasn't generalized the data patterns correctly. It was only replicating the data distribution in a highly specialized or generic way.

![](https://www.mathworks.com/discovery/overfitting/_jcr_content/mainParsys/image.adapt.full.medium.svg/1705396624275.svg)

## Train-validation-test split

One way to try to cope with overfitting is doing a more complex strategy for dividing the dataset into:

1. Train datased: used to train the model
1. Validation (or development) dataset: used to optimize our model parameters to achieve best accuracy possible
1. Test dataset: unseen data separated to test our model with possibly different patterns not found in previous datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42
)

# F-strings https://realpython.com/python-f-strings/
print(f'Training size: {len(X_train)}, Validation size: {len(X_dev)}, Test size: {len(X_test)}')

In [ ]:
# Create the model
dtree = DecisionTreeClassifier()

# Train the model
dtree.fit(X_train, y_train)

In [ ]:
viz = dtreeviz.model(dtree, X_train, y_train,
               target_name="Species",
               feature_names=iris.feature_names,
               class_names=list(iris.target_names))

viz.view(fontname="DejaVu Sans")

## Checking accuracy with the validation dataset

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_dev_predicted = dtree.predict(X_dev)
y_dev_predicted

In [ ]:
print(f'Accuracy with validation dataset {accuracy_score(y_dev_predicted, y_predicted)}')

At this point we can do multiple runs of train/validation checks to improve the accuracy score...

## Inferring over new unforeseen data (X_test)

In [ ]:
# The same but with X_test, y_test
y_test_predicted = dtree.predict(X_test)
y_test_predicted

In [ ]:
print(f'Accuracy with test dataset {accuracy_score(y_test_predicted, y_predicted)}')

## Decision trees for regression problems

### Diabetes dataset

Target is a quantitative measure of disease progression one year after baseline

![](https://explained.ai/decision-tree-viz/images/samples/diabetes-TD-3-X.svg)

In the above example we can see Decision Trees create **LINEAR** decision boundaries for its variables.
This is amazing to generate (visual) explanations for your resulting model BUT...

What would happen if the feature vector distribution is like in the following image:
![](https://media.geeksforgeeks.org/wp-content/uploads/20200605170732/linearsep.png)

In this case we probably need Support Vector Machines (SVM), which are able to divide the feature space into curved decision boundaries.

## Additional reading

- [A visual introduction to machine learning](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
- [Understanding Decision Trees with Python](https://datascience.foundation/sciencewhitepaper/understanding-decision-trees-with-python)
- [4 Ways to Visualize Individual Decision Trees in a Random Forest](https://towardsdatascience.com/4-ways-to-visualize-individual-decision-trees-in-a-random-forest-7a9beda1d1b7)
- [How to Visualize a Random Forest with Fitted Parameters?](https://analyticsindiamag.com/how-to-visualize-a-random-forest-with-fitted-parameters/)
- [Understanding Random forest better through visualizations](https://garg-mohit851.medium.com/random-forest-visualization-3f76cdf6456f)

## Improving the Model

There are two very useful techniques that allow us to create new models:

* **Bootstrap**: It is based on choosing subsamples of our data in a uniform way and with repetition, thus creating multiple smaller samples that share the distributions of the original sample.

* **Bagging**: Generate a bootstrap of size $n$, train a model on that subsample and repeat the process $m$ times.

## Random Forests

Now let's apply an *ensemble* method such as *Random Forest*, based on the results of multiple decision trees.



There are other types of combining classifiers (or ensemble methods), such as mean average, or the product rule, using a priory probability distributions and the confidence of the classification of each classifier.

Additional reading:

- [Scikit Learn ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html)

## Training a Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=100)

rf = RandomForestClassifier(n_jobs=-1) # parallelize the execution
rf.fit(X_train, y_train)

Let's analyze the code ... ([RFs in Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html))

In [ ]:
help(rf)

## Real species

In [ ]:
y_test

## Predicting the class of test dataset

In [ ]:
predicted = rf.predict(X_test)
predicted

## Accuracy score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
accuracy_score(y_test, predicted)

In [ ]:
help(accuracy_score)

## Feature importance

In [ ]:
rf = RandomForestClassifier(n_jobs=4)
rf

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
importances = rf.feature_importances_
importances

In [ ]:
indexes = np.argsort(importances)[::-1]
indexes

In [ ]:
X.columns

In [ ]:
# Map importances to species_names
names = [X.columns[i] for i in indexes]
names

In [ ]:
# Prepare a barplot
plt.bar(range(X_train.shape[1]), importances[indexes])
# Add the feature names
plt.xticks(range(X_train.shape[1]), names, rotation=20, fontsize = 8)
# Add the title
plt.title("Feature Importance")
# Self explanatory...
plt.show()

In the code of the upper cell we see not only how we can apply a model, but that it contains the importance of the different *features* or predictor variables.

## Predict() function and confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

rf_preds = rf.predict(X_test)
rf_conf_mat = confusion_matrix(y_test, rf_preds)
rf_conf_mat

In [ ]:
help(confusion_matrix)

You can get the accuracy score with data from confusion matrix

In [ ]:
# Convert to numpy
np_mat = np.asarray(rf_conf_mat)

acc = sum(np.diagonal(np_mat)) / np_mat.sum()
print(f"My accuracy is: {acc}")

## Confusion matrix visualization

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=rf_conf_mat,display_labels=iris.target_names)
disp.plot()